In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [2]:
# Option 1: 
# Use a CW equal to the 3 panels, and replicate 3 times along the row.
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo3'
epwfile = r'C:\Users\sayala\Documents\RadianceScenes\Demo3\EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw'
# This is the only thing that changes for running on the HPC at the moment
#testfolder = '/scratch/sayala/RadianceScenes/Test1'
#epwfile='/scratch/sayala/EPWs/USA_VA_Richmond.Intl.AP.724010_TMY.epw'

simulationname = 'TestHPC1'
moduletype = 'Longi'

# Control Variables
cumulativesky=False
rewriteModule=True
hpc=True

# Scene Variables 
gcr = 0.35   # ground cover ratio,  = module_height / pitch
albedo = 0.3     # ground albedo
nMods = 20  # replicating the unit 3 times
nRows = 7  # only 1 row

# Analysis Variabls
sensorsy = 210  # this will give 70 sensors per module.
modWanted = nMods/2
rowWanted = nRows/2

# Fixed tilt variables
azimuth_ang=180 # Facing south
tilt =20 # tilt. 

# Tracker geometry options
backtrack = True
hub_height = 2   # tracker height at 0 tilt in meters (hub height)
limit_angle = 45 # tracker rotation limit angle
axis_azimuth=180
angledelta=5


# ModuleDict Variables
numpanels = 1
x=0.984  
y = 1.996
bifi=0.9
xgap = 0.01 # 10 centimeters.
ygap = 0.0
zgap = 0.1

# TorqueTube Parameters
torqueTube = True
axisofrotationTorqueTube=False
diameter=0.1
tubetype='Round'
torqueTubeMaterial='Metal_Grey'

# Cell Level Module Parameters
cellLevelModule = False
numcellsx=10
numcellsy=6
xcell=0.156
ycell=0.156
xcellgap=0.02
ycellgap=0.02


In [3]:
demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,bifi=bifi, 
                       torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                       material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                       rewriteModulefile = True, xgap=xgap, 
                       axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                       numcellsx=numcellsx, numcellsy = numcellsy)
sceneDict = {'module_type':moduletype, 'pitch': round(moduleDict['sceney'] / gcr,3),'height':hub_height, 'nMods':nMods, 'nRows':nRows} #, 'nMods':20, 'nRows':7}  

metdata = demo.readEPW(epwfile, hpc=True, daydate='05_11') # read in the EPW weather data from above

trackerdict = demo.set1axis(cumulativesky=cumulativesky, axis_azimuth=axis_azimuth, limit_angle=limit_angle, angledelta=angledelta, backtrack=backtrack, gcr=gcr)
trackerdict = demo.gendaylit1axis(hpc=hpc)  #benchmark time: gendaylit2:105s.  gendaylit: 5s

# Use this to simulate only one hour at a time. 
# This allows you to "view" the scene on RVU (see instructions below)
# timestam 4020 : Noon, June 17th.
#demo.genCumSky(demo.epwfile) # Use this for simulate the whole year



path = C:\Users\sayala\Documents\RadianceScenes\Demo3
('\nModule Name:', 'Longi')
Module file did not exist before, creating new module file
Module Longi successfully created
Creating ~4000 skyfiles.  Takes 1-2 minutes
Adjusting solarposition for sunrise hour, 0 timeindex
Original datetime 1974-05-11 06:00:00-05:00
Localized and adjusted datetime 1974-05-11 05:32:00-05:00 

Created 14 skyfiles in /skies/


In [4]:

trackerdict

{'05_11_06': {'dhi': 15L,
  'ghi': 15L,
  'skyfile': 'skies\\sky2_37.5_-77.33_05_11_06.rad',
  'surf_azm': 90.0,
  'surf_tilt': 7.919520706149697,
  'theta': -7.9195207061497115},
 '05_11_07': {'dhi': 81L,
  'ghi': 82L,
  'skyfile': 'skies\\sky2_37.5_-77.33_05_11_07.rad',
  'surf_azm': 90.0,
  'surf_tilt': 35.04244331245067,
  'theta': -35.042443312450665},
 '05_11_08': {'dhi': 193L,
  'ghi': 210L,
  'skyfile': 'skies\\sky2_37.5_-77.33_05_11_08.rad',
  'surf_azm': 90.0,
  'surf_tilt': 44.99999999999999,
  'theta': -45.0},
 '05_11_09': {'dhi': 311L,
  'ghi': 382L,
  'skyfile': 'skies\\sky2_37.5_-77.33_05_11_09.rad',
  'surf_azm': 90.0,
  'surf_tilt': 44.99999999999999,
  'theta': -45.0},
 '05_11_10': {'dhi': 307L,
  'ghi': 575L,
  'skyfile': 'skies\\sky2_37.5_-77.33_05_11_10.rad',
  'surf_azm': 90.0,
  'surf_tilt': 37.6677912024576,
  'theta': -37.66779120245761},
 '05_11_11': {'dhi': 408L,
  'ghi': 545L,
  'skyfile': 'skies\\sky2_37.5_-77.33_05_11_11.rad',
  'surf_azm': 90.0,
  'surf_t

In [5]:
trackerdict = demo.makeScene1axis(trackerdict=trackerdict, moduletype=moduletype, sceneDict=sceneDict, cumulativesky=cumulativesky) #makeScene creates a .rad file with 20 modules per row, 7 rows.
demo.makeOct1axis(trackerdict, hpc=True)



Making ~4000 .rad files for gendaylit 1-axis workflow (this takes a minute..)
14 Radfiles created in /objects/

Making 14 octfiles for 1-axis tracking in root directory.
Created 1axis_05_11_16.oct
Created 1axis_05_11_17.oct
Created 1axis_05_11_14.oct
Created 1axis_05_11_15.oct
Created 1axis_05_11_12.oct
Created 1axis_05_11_13.oct
Created 1axis_05_11_10.oct
Created 1axis_05_11_11.oct
Created 1axis_05_11_18.oct
Created 1axis_05_11_19.oct
Created 1axis_05_11_09.oct
Created 1axis_05_11_08.oct
Created 1axis_05_11_07.oct
Created 1axis_05_11_06.oct


{'05_11_06': {'dhi': 15L,
  'ghi': 15L,
  'ground_clearance': 1.8624935587937745,
  'octfile': '1axis_05_11_06.oct',
  'radfile': 'objects\\1axis05_11_06_1.862_5.703_20x7.rad',
  'scene': <bifacial_radiance.bifacial_radiance.SceneObj instance at 0x000000000D1E0E48>,
  'skyfile': 'skies\\sky2_37.5_-77.33_05_11_06.rad',
  'surf_azm': 90.0,
  'surf_tilt': 7.919520706149697,
  'theta': -7.9195207061497115},
 '05_11_07': {'dhi': 81L,
  'ghi': 82L,
  'ground_clearance': 1.4269652794482024,
  'octfile': '1axis_05_11_07.oct',
  'radfile': 'objects\\1axis05_11_07_1.426_5.703_20x7.rad',
  'scene': <bifacial_radiance.bifacial_radiance.SceneObj instance at 0x000000000D26F148>,
  'skyfile': 'skies\\sky2_37.5_-77.33_05_11_07.rad',
  'surf_azm': 90.0,
  'surf_tilt': 35.04244331245067,
  'theta': -35.042443312450665},
 '05_11_08': {'dhi': 193L,
  'ghi': 210L,
  'ground_clearance': 1.2943074323758257,
  'octfile': '1axis_05_11_08.oct',
  'radfile': 'objects\\1axis05_11_08_1.294_5.703_20x7.rad',
  'scen

In [6]:
trackerdict

{'05_11_06': {'dhi': 15L,
  'ghi': 15L,
  'ground_clearance': 1.8624935587937745,
  'octfile': '1axis_05_11_06.oct',
  'radfile': 'objects\\1axis05_11_06_1.862_5.703_20x7.rad',
  'scene': <bifacial_radiance.bifacial_radiance.SceneObj instance at 0x000000000D1E0E48>,
  'skyfile': 'skies\\sky2_37.5_-77.33_05_11_06.rad',
  'surf_azm': 90.0,
  'surf_tilt': 7.919520706149697,
  'theta': -7.9195207061497115},
 '05_11_07': {'dhi': 81L,
  'ghi': 82L,
  'ground_clearance': 1.4269652794482024,
  'octfile': '1axis_05_11_07.oct',
  'radfile': 'objects\\1axis05_11_07_1.426_5.703_20x7.rad',
  'scene': <bifacial_radiance.bifacial_radiance.SceneObj instance at 0x000000000D26F148>,
  'skyfile': 'skies\\sky2_37.5_-77.33_05_11_07.rad',
  'surf_azm': 90.0,
  'surf_tilt': 35.04244331245067,
  'theta': -35.042443312450665},
 '05_11_08': {'dhi': 193L,
  'ghi': 210L,
  'ground_clearance': 1.2943074323758257,
  'octfile': '1axis_05_11_08.oct',
  'radfile': 'objects\\1axis05_11_08_1.294_5.703_20x7.rad',
  'scen

In [7]:
trackerdict = demo.analysis1axis(trackerdict, sceneDict=sceneDict, modWanted=modWanted, rowWanted=rowWanted, sensorsy=sensorsy)


Linescan in process: 1axis_05_11_06_Front
Linescan in process: 1axis_05_11_06_Back
Saved: results\irr_1axis_05_11_06.csv
Index: 05_11_06. Wm2Front: 14.811407873. Wm2Back: 2.68970144286
Linescan in process: 1axis_05_11_07_Front
Linescan in process: 1axis_05_11_07_Back
Saved: results\irr_1axis_05_11_07.csv
Index: 05_11_07. Wm2Front: 78.8491492222. Wm2Back: 16.6236460794
Linescan in process: 1axis_05_11_08_Front
Linescan in process: 1axis_05_11_08_Back
Saved: results\irr_1axis_05_11_08.csv
Index: 05_11_08. Wm2Front: 272.331610476. Wm2Back: 41.1414617619
Linescan in process: 1axis_05_11_09_Front
Linescan in process: 1axis_05_11_09_Back
Saved: results\irr_1axis_05_11_09.csv
Index: 05_11_09. Wm2Front: 479.720128889. Wm2Back: 71.1063296825
Linescan in process: 1axis_05_11_10_Front
Linescan in process: 1axis_05_11_10_Back
Saved: results\irr_1axis_05_11_10.csv
Index: 05_11_10. Wm2Front: 707.739316508. Wm2Back: 95.0143205714
Linescan in process: 1axis_05_11_11_Front
Linescan in process: 1axis_05

In [15]:
import os
radfile = os.path.join(testfolder,'objects','%s_%0.5s_%0.5s_%sx%s.rad'%('sasdf',2.3,3, 7, 4) ) # update in 0.2.3 to shorten radnames
radfile

'C:\\Users\\sayala\\Documents\\RadianceScenes\\Demo3\\objects\\sasdf_2.3_3_7x4.rad'

In [21]:
radfile

'C:\\Users\\sayala\\Documents\\RadianceScenes\\Demo3\\objects\\sasdf_2.3_3_7x4.rad'

In [29]:
trackerdict['05_11_06']['radfile']



'objects\\1axis05_11_06_1.862_5.703_20x7.rad'